In [ ]:
import pandas as pd
from dhis2 import Api

from src.main.common import FileUtil
from src.main.application.income import *
from src.main.application.service import *
from src.infrastructure.forms import *
from src.infrastructure.adapters import *

In [ ]:
credentials  = FileUtil.load_credentias()
username = credentials['username']
password = credentials['password']

url = 'https://dhis-ao.icap.columbia.edu'
api = Api(url, username, password)

In [ ]:
# Get the list of supported facilities
org_unit = 'UnCWi0cmwXy'

start_period = '2024-01-01'
end_period = '2024-03-31'

In [ ]:
#get all patient enrollments
patients = api.get('tracker/enrollments', params={'orgUnit':org_unit, 'skipPaging':'true', 'program': CARE_AND_TREATMENT, 'fields':'{,enrollment, enrolledAt, trackedEntity, program, status,}'})
patients = patients.json()['instances']

for patient in patients:
    patient_id = patient['trackedEntity']

    patient_demographics = PatientDemographicForm(patient_id, org_unit, api)
    patient_demographics.add_demographics(patient)

    patient_consultation= PatientConsultationForm(patient_id, org_unit, api)
    patient_consultation.add_first_consultation(patient)

    patient_pharmacy_form =  PatientPharmacyForm(patient_id, org_unit, api)
    patient_pharmacy_form.add_last_pharmacy(patient)

    patient_laboratory_from =  PatientLaboratoryForm(patient_id, org_unit, api)
    patient_laboratory_from.add_laboratory(patient)

    patient_outcome_form =  PatientOutcomeForm(patient_id, org_unit, api)
    patient_outcome_form.add_final_outcome(patient)

In [ ]:
tx_curr_service = ComputeTxCurrService()
tx_curr_service.compute(patients, end_period)

tx_new = ComputeTxNewService(tx_curr_service)
tx_new.compute(patients,start_period,end_period)

tx_ml = ComputeTxMlService()
tx_ml.compute(patients, start_period, end_period)

laboratory_port = LaboratoryAdapter(patient_laboratory_from)
tx_pvls_denominator = ComputeTxPvlsDenominatorService(tx_curr_service, laboratory_port)
tx_pvls_denominator.compute(patients, end_period)

tx_pvls_numerator = ComputeTxPvlsNumeratorService()
tx_pvls_numerator.compute(patients)

In [ ]:
data = pd.json_normalize(patients)
data.to_excel('DATA.xlsx', index=False)